In [55]:
import numpy as np
import pandas as pd
import glob
import argparse
import os
from pathlib import Path

import os

def joinfile(*args):
    return os.path.normpath(os.path.join(*args))

pd.set_option('display.max_colwidth', None)

mapping_path = '/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/data/raw/tables/stn_space/merged/flipped/mapping/multicentricTableBernCologneImprovedOnlyRev05Patient21Excluded.csv'
output_folder = '/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/data/raw/tables/stn_space'
#vtas_path = '/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/02g_MassiveMerged'

In [56]:
def set_interp(df : pd.DataFrame):
    # linear interpolation 
    #print()
    #print(name, end=' : ')
    groups = df.groupby(['leadID', 'contactID'])
    for name, group in groups:
        #print(name)
        #print(group[['amplitude', 'mapping']])
        # if there are mapping values in the group
        # [2,4] -> [0.17, 0.33]
        # excepted : [1, 2, 3, 4, 5] -> [0.085, 0.17, 0.25, 0.33, 0.41]
        if not group[group['mapping'] == 1].empty:
            first_amp = group[group['mapping'] == 1]['amplitude'].to_list()[0]
            score = group[group['mapping'] == 1]['relativeImprovement'].to_list()[0]
            #print(max_amp, end = '')
                        
            # max amplitude at 3 -> score = 0.6
            xp = (0., first_amp)
            fp = (0., score)
            amplitudes_interp = group[group['amplitude'] <= first_amp]['amplitude'].to_list()
            amplitudes_left = group[group['amplitude'] > first_amp]['amplitude'].to_list()
            #print(amplitudes)
            interp = np.interp(amplitudes_interp, xp, fp)
            interp = np.append(interp, amplitudes_left)
            #print(interp)
            #print(df.loc[((df['leadID'] == name[0]) & ([df['contactID'] == name[1]])),'relativeImprovement'])
            #break
            a = df['leadID'] == name[0]
            b = df['contactID'] == name[1]
            df.loc[a & b,'relativeImprovement'] = interp
            #print(df.loc[a & b,'relativeImprovement'])
            #group['relativeImprovement'] = interp
            
            #print(df.query(f'leadID == {name[0]} and contactID == {name[1]}')['relativeImprovement'])
            
            #print(group[['amplitude', 'relativeImprovement', 'mapping']])
    return df


def set_step(df : pd.DataFrame):
    #print()
    #print(name, end=' : ')
    #print(group[['amplitude', 'mapping']])
    groups = df.groupby(['leadID', 'contactID'])
    for name, group in groups:
        if not group[group['mapping'] == 1].empty:
            a = df['leadID'] == name[0]
            b = df['contactID'] == name[1]
            df.loc[a & b,'relativeImprovement'] = group['relativeImprovement'].interpolate(method='pad')

        #print(group[['amplitude', 'relativeImprovement', 'mapping']])
    df['relativeImprovement'] = df['relativeImprovement'].fillna(0.0)
    return df

def get_idx_to_del(df : pd.DataFrame):
    idx_to_del_part, idx_to_del_full = [], []
    groups = df.groupby(['leadID', 'contactID'])
    for name, group in groups:
        #print()
        #print(name, end=' : ')
        #print(group[['amplitude', 'mapping']])
        if group[group['mapping'] == 1].empty:
            idx_buff = group.index
            idx_to_del_full.extend(idx_buff)
            idx_to_del_part.extend(idx_buff)
            #print('no mapping here', end = '')
            #print(group_.index)
        else:
            max_amp = group[group['mapping'] == 1]['amplitude'].to_list()

            idx_buff_full = group[(group['amplitude'].isin(max_amp)) & (group['mapping'] == 0)].index
            idx_to_del_full.extend(idx_buff_full)
            
            idx_buff_part = group[(group['amplitude'] >= max(max_amp)) & (group['mapping'] == 0)].index
            idx_to_del_part.extend(idx_buff_part)
            #print(idx_buff_part)
    return idx_to_del_part, idx_to_del_full


In [57]:
# --- GETTING ALL THE FILEPATHS IN THIS FORMAT AND MERGING LEFT & RIGHT ---
flipped_list = glob.glob('/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/*.nii')  
#print(flipped_list)
# --- DICT TO CONVERT CONTACT LEAD NAMING CONVERTIONS ---------------------
medtronic_to_leaddbs = {
    '0' : '0',
    '1' : '16',
    '2' : '17',
    '3' : '7',
    '8' : '8',
    '9' : '18',
    '10' : '19',
    '11' : '15'
}

leaddbs_to_medtronic = {
    '0' : '0',
    '16' : '1',
    '17' : '2',
    '7' : '3',
    '8' : '8',
    '18' : '9',
    '19' : '10',
    '15' : '11'
}

vercise_to_ring = {
     '0' : 'R0',
     '16' : 'R1',
     '17' : 'R2',
     '7' : 'R3',
     '8' : 'R0',
     '18' : 'R1',
     '19' : 'R2',
     '15' : 'R3'
}

vercise_to_dir = {
     '1' : 'R1_1',
     '2' : 'R1_2',
     '3' : 'R1_3',
     '4' : 'R2_1',
     '5' : 'R2_2',
     '6' : 'R2_3',
     '9' : 'R1_1',
     '10' : 'R1_2',
     '11' : 'R1_3',
     '12' : 'R2_1',
     '13' : 'R2_2',
     '14' : 'R2_3',
}

b01 = (1, 3, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29)
b02 = (31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 82, 119, 134, 138, 149, 175, 176, 177, 178, 179, 180, 181, 183)
c01 = (201, 202, 203, 204, 205, 206, 207, 208, 211, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225)
c02 = (231, 232, 233, 234, 235, 236, 237, 238, 239, 240)

print(f'items : {len(flipped_list)}')
flipped_list = [x for x in flipped_list if 'p221' not in x.split('/')[-1].split('_')[0]] #excluding patient 21
print(f'items after excluding patient 21 : {len(flipped_list)}')
# --- MAKING ROW ENTRIES ACCORDING TO leaddbs SAVARY'S PSM CLASS ----------
centerID, contactID, leadID, verciseID, score, amplitude, leadModel, patientID, massive_filename, hemisphere = [], [], [], [], [], [], [], [], [], []
cohort, directional, rings, segmented = [], [], [], []
p221 = 0
zeroed = 0
for item in flipped_list:
    #print(item)
    if 'zeroed' in item:
         zeroed += 1
         continue
    # '/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/02g_MassiveMerged/p003_c12_a4.nii'
    patient = int(item.split('/')[-1].split('_')[0][1:])
    if patient == 221:
            p221 += 1
            continue
    elif patient in b01:
         boston = True
         leadModel.append('B.S. Vercise')
         cohort.append('B01')
         centerID.append('Bern') 
    elif patient in b02: 
        boston = True
        leadModel.append('B.S. Vercise')
        cohort.append('B02')
        centerID.append('Bern') 
    elif patient in c01: # p200-229
        boston = False
        leadModel.append('M. 3389')
        cohort.append('C01')
        centerID.append('Cologne')   
    elif patient in c02: 
        boston = True
        leadModel.append('B.S. Vercise')
        cohort.append('C02')
        centerID.append('Cologne') 
    else:
         raise ValueError(f"{patient} not in cohorts list")
    patientID.append(str(patient))
    vercise = item.split('/')[-1].split('_')[1][1:]

    if vercise in vercise_to_ring.keys():
        directional.append(False)
        rings.append(vercise_to_ring[vercise])
    else:
        directional.append(True)
        rings.append('')

    if vercise in vercise_to_dir.keys():
        segmented.append(vercise_to_dir[vercise])
    else:
        segmented.append('')

    # vercise stores the contact ID of all model corresponding to BS Vercise model and the lead-dbs nomenclature
    if not boston:
        contact = leaddbs_to_medtronic[vercise]
    else:
        contact = vercise
    
    side = 'right' if int(vercise) in (0,1,2,3,4,5,6,7,16,17) else 'left'
    hemisphere.append(side)

    if side == 'left':
         leadID.append(f"{str(int(patient))}.5")
    else:
         leadID.append(f"{str(int(patient))}.0")
    #contact = contact_flipped_to_left[contact] if hemisphere == 'right' else contact
    contactID.append(int(contact)) #smaller = left, bigger = right
    verciseID.append(int(vercise))
    

    amplitude_ = item.split('/')[-1].split('_')[2][1:-4]
    amplitude.append(float(amplitude_)) #in name

    massive_filename.append(item)

print(f"inferred length : {len(massive_filename)}, zeroed : {zeroed}, p221 : {p221}")
#print(len(centerID), len(leadModel), len(contactID), len(relativeImprovement), len(amplitude), len(efficiency), len(flipped_list))
dict_infer = {
    'center' : centerID,
    'cohort' : cohort,
    'lead_model' : leadModel,
    'patient' : patientID,
    'lead' : leadID,
    'hemisphere' : hemisphere,
    'contact' : contactID,
    'vercise' : verciseID,
    'directional' : directional,
    'ring' : rings,
    'segmented' : segmented,
    'amplitude' : amplitude,
    'massive_filename' : massive_filename}
df_infer = pd.DataFrame(dict_infer, index = list(range(len(massive_filename))))


items : 14157
items after excluding patient 21 : 14157
inferred length : 14157, zeroed : 0, p221 : 0


In [58]:
df_infer.columns

Index(['center', 'cohort', 'lead_model', 'patient', 'lead', 'hemisphere',
       'contact', 'vercise', 'directional', 'ring', 'segmented', 'amplitude',
       'massive_filename'],
      dtype='object')

In [59]:
df_infer.head(20)

,center,cohort,lead_model,patient,lead,hemisphere,contact,vercise,directional,ring,segmented,amplitude,massive_filename
0,Bern,B02,B.S. Vercise,119,119.5,left,11,11,True,,R1_3,4.5,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p119_c11_a4.5.nii
1,Cologne,C01,M. 3389,223,223.5,left,9,18,False,R1,,1.5,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p223_c18_a1.5.nii
2,Bern,B02,B.S. Vercise,178,178.0,right,5,5,True,,R2_2,2.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p178_c5_a2.nii
3,Bern,B01,B.S. Vercise,1,1.5,left,9,9,True,,R1_1,2.5,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c9_a2.5.nii
4,Bern,B02,B.S. Vercise,36,36.5,left,13,13,True,,R2_2,6.5,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p036_c13_a6.5.nii
5,Cologne,C02,B.S. Vercise,237,237.0,right,1,1,True,,R1_1,1.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p237_c1_a1.nii
6,Cologne,C01,M. 3389,218,218.0,right,3,7,False,R3,,3.5,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p218_c7_a3.5.nii
7,Bern,B02,B.S. Vercise,34,34.5,left,10,10,True,,R1_2,4.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p034_c10_a4.nii
8,Cologne,C02,B.S. Vercise,239,239.5,left,8,8,False,R0,,4.5,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p239_c8_a4.5.nii
9,Bern,B01,B.S. Vercise,9,9.0,right,1,1,True,,R1_1,2.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p009_c1_a2.nii


In [60]:
print(len(df_infer))

14157


In [61]:
# # --- EXPORTING FULL INFERRED TABLE ---------------------------------------
# output_path = joinfile(output_folder, 'inferred/nan_interp/table_psm.csv')
# Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
# df_infer.to_csv(output_path)

df_infer['mapping'] = 0
output_path = joinfile(output_folder, 'inferred/table.csv')
Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
df_infer.to_csv(output_path)


In [62]:
df_infer.dtypes

center               object
cohort               object
lead_model           object
patient              object
lead                 object
hemisphere           object
contact               int64
vercise               int64
directional            bool
ring                 object
segmented            object
amplitude           float64
massive_filename     object
mapping               int64
dtype: object

In [63]:
# --- IMPORTING MAPPING TABLE ---------------------------------------------
df_map = pd.read_csv(mapping_path)
center_dict = {
    1 : 'Bern',
    2 : 'Cologne'
}

def assign_lead_model(row):
    if int(row['patient']) < 200 or int(row['patient']) > 229:
        return 'B.S. Vercise'
    else:
        return 'M. 3389'
    
def assign_vercise_id(row):
    if row['lead_model'] == 'B.S. Vercise':
        return row['contact']
    else:
        return medtronic_to_leaddbs[row['contact']]

def assign_cohort(row):
    if int(row['patient']) in b01:
         return 'B01'

    elif int(row['patient']) in b02: 
        return 'B02'

    elif int(row['patient']) in c01: # p200-229
        return 'C01'

    elif int(row['patient']) in c02: 
        return 'C02'

    else:
         raise ValueError(f"{patient} not in cohorts list")
    
def assign_hemisphere(row):
    if '.5' in row['lead'] :
        return 'left'
    else:
        return 'right'
    
def assign_directional(row):
    if row['vercise'] in vercise_to_ring.keys():
        return False
    else:
        return True

def assign_ring(row):
    if row['vercise'] in vercise_to_ring.keys():
        return vercise_to_ring[row['vercise']]
    else:
        return ''

def assign_segmented(row):
    if row['vercise'] in vercise_to_dir.keys():
        return vercise_to_dir[row['vercise']]
    else:
        return ''

def old_to_new_filename(filename):
    massive_folder = '/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/'
    patient = 'p' + str(int(filename.split('/')[-1].split('_')[0])).zfill(3)
    contact = str(filename.split('/')[-1].split('_')[1])
    patient_id = int(filename.split('/')[-1].split('_')[0])
    if patient_id >= 201 and patient_id < 230:
        contact = medtronic_to_leaddbs[contact]
    contact = '_c' + contact
    amplitude = '_a' + filename.split('/')[-1].split('_')[2][:-4] + '.nii'
    new_filename = massive_folder + patient + contact + amplitude
    return new_filename

df_map

,centerID,leadID,contactID,relativeImprovement,amplitude,efficiency,filename
0,1,1.0,0,0.333333,4.5,0.074074,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_0_4.5.nii
1,1,1.0,1,1.000000,3.0,0.333333,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_1_3.nii
2,1,1.0,16,1.000000,4.0,0.250000,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_16_4.nii
3,1,1.0,2,1.000000,3.5,0.285714,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_2_3.5.nii
4,1,1.0,3,1.000000,2.5,0.400000,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_3_2.5.nii
...,...,...,...,...,...,...,...
999,2,240.0,3,0.500000,3.0,0.166667,../../03_Data/VTAsForMulticentricPSMRev04B/CologneVTAsResliced/240_3_3.nii
1000,2,240.0,3,0.500000,4.0,0.125000,../../03_Data/VTAsForMulticentricPSMRev04B/CologneVTAsResliced/240_3_4.nii
1001,2,240.0,2,0.333333,2.0,0.166667,../../03_Data/VTAsForMulticentricPSMRev04B/CologneVTAsResliced/240_2_2.nii
1002,2,240.0,2,0.333333,3.0,0.111111,../../03_Data/VTAsForMulticentricPSMRev04B/CologneVTAsResliced/240_2_3.nii


In [64]:
df_map = df_map.drop(columns=('efficiency'))

df_map['center'] = df_map['centerID'].map(center_dict)
df_map = df_map.drop(columns=('centerID'))

df_map = df_map.rename(columns={'contactID' : 'contact', 'centerID' : 'center', 'leadID' : 'lead'})

df_map['patient'] = df_map['lead'].map(lambda x : str(int(x)))

df_map['lead'] = df_map['lead'].astype(str)

df_map['contact'] = df_map['contact'].astype(str)

df_map = df_map.rename(columns={'relativeImprovement' : 'mapping_score', 'filename' : 'old_filename'})

df_map['massive_filename'] = df_map['old_filename'].map(old_to_new_filename)

df_map['lead_model'] = df_map.apply(assign_lead_model, axis=1)
df_map['vercise'] = df_map.apply(assign_vercise_id, axis=1)
df_map['cohort'] = df_map.apply(assign_cohort, axis=1)
df_map['hemisphere'] = df_map.apply(assign_hemisphere, axis=1)
df_map['segmented'] = df_map.apply(assign_segmented, axis=1)
df_map['ring'] = df_map.apply(assign_ring, axis=1)
df_map['directional'] = df_map.apply(assign_directional, axis=1)

df_map['mapping'] = 1
df_map = df_map[['center', 'cohort', 'lead_model', 'patient', 'lead', 'hemisphere',
       'contact', 'vercise', 'directional', 'ring', 'segmented', 'amplitude', 'mapping_score', 'mapping',
       'massive_filename', 'old_filename']]
#df_map.to_csv('mapping_merged.csv', index=False)
df_map

,center,cohort,lead_model,patient,lead,hemisphere,contact,vercise,directional,ring,segmented,amplitude,mapping_score,mapping,massive_filename,old_filename
0,Bern,B01,B.S. Vercise,1,1.0,right,0,0,False,R0,,4.5,0.333333,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c0_a4.5.nii,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_0_4.5.nii
1,Bern,B01,B.S. Vercise,1,1.0,right,1,1,True,,R1_1,3.0,1.000000,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c1_a3.nii,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_1_3.nii
2,Bern,B01,B.S. Vercise,1,1.0,right,16,16,False,R1,,4.0,1.000000,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c16_a4.nii,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_16_4.nii
3,Bern,B01,B.S. Vercise,1,1.0,right,2,2,True,,R1_2,3.5,1.000000,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c2_a3.5.nii,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_2_3.5.nii
4,Bern,B01,B.S. Vercise,1,1.0,right,3,3,True,,R1_3,2.5,1.000000,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c3_a2.5.nii,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_3_2.5.nii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,Cologne,C02,B.S. Vercise,240,240.0,right,3,3,True,,R1_3,3.0,0.500000,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p240_c3_a3.nii,../../03_Data/VTAsForMulticentricPSMRev04B/CologneVTAsResliced/240_3_3.nii
1000,Cologne,C02,B.S. Vercise,240,240.0,right,3,3,True,,R1_3,4.0,0.500000,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p240_c3_a4.nii,../../03_Data/VTAsForMulticentricPSMRev04B/CologneVTAsResliced/240_3_4.nii
1001,Cologne,C02,B.S. Vercise,240,240.0,right,2,2,True,,R1_2,2.0,0.333333,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p240_c2_a2.nii,../../03_Data/VTAsForMulticentricPSMRev04B/CologneVTAsResliced/240_2_2.nii
1002,Cologne,C02,B.S. Vercise,240,240.0,right,2,2,True,,R1_2,3.0,0.333333,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p240_c2_a3.nii,../../03_Data/VTAsForMulticentricPSMRev04B/CologneVTAsResliced/240_2_3.nii


In [65]:
df_map.dtypes

center               object
cohort               object
lead_model           object
patient              object
lead                 object
hemisphere           object
contact              object
vercise              object
directional            bool
ring                 object
segmented            object
amplitude           float64
mapping_score       float64
mapping               int64
massive_filename     object
old_filename         object
dtype: object

In [66]:
len(df_infer), len(df_map)

(14157, 1004)

In [67]:
# --- ERASING ENTRIES ON INFERRED TABLE -----------------------------------
# Convert the data types of the merging columns to be the same

columns = ['patient', 'contact', 'amplitude', 'center', 'cohort', 'ring', 'segmented', 'vercise', 'hemisphere']
df_map[columns] = df_map[columns].astype(str)


df_infer[columns] = df_infer[columns].astype(str)


# Perform an outer merge
df_merged = pd.merge(df_map, df_infer, how='outer')

# Prioritize df_map over df_infer for overlapping rows
df_merged = df_map.combine_first(df_merged)
df_merged

,center,cohort,lead_model,patient,lead,hemisphere,contact,vercise,directional,ring,segmented,amplitude,mapping_score,mapping,massive_filename,old_filename
0,Bern,B01,B.S. Vercise,1,1.0,right,0,0,False,R0,,4.5,0.333333,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c0_a4.5.nii,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_0_4.5.nii
1,Bern,B01,B.S. Vercise,1,1.0,right,1,1,True,,R1_1,3.0,1.000000,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c1_a3.nii,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_1_3.nii
2,Bern,B01,B.S. Vercise,1,1.0,right,16,16,False,R1,,4.0,1.000000,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c16_a4.nii,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_16_4.nii
3,Bern,B01,B.S. Vercise,1,1.0,right,2,2,True,,R1_2,3.5,1.000000,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c2_a3.5.nii,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_2_3.5.nii
4,Bern,B01,B.S. Vercise,1,1.0,right,3,3,True,,R1_3,2.5,1.000000,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c3_a2.5.nii,../../03_Data/VTAsForMulticentricPSMRev05/BernVTAsResliced/0001_3_2.5.nii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15156,Bern,B01,B.S. Vercise,12,12.0,right,2,2,True,,R1_2,3.5,NaN,0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p012_c2_a3.5.nii,NaN
15157,Bern,B02,B.S. Vercise,31,31.0,right,17,17,False,R2,,5.0,NaN,0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p031_c17_a5.nii,NaN
15158,Cologne,C02,B.S. Vercise,237,237.0,right,3,3,True,,R1_3,0.5,NaN,0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p237_c3_a0.5.nii,NaN
15159,Cologne,C01,M. 3389,205,205.0,right,1,16,False,R1,,0.5,NaN,0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p205_c16_a0.5.nii,NaN


In [68]:

import math
print(len(df_merged))
df_merged = df_merged.drop(columns=['old_filename'])
#df_merged = df_merged.astype({'amplitude': 'float64'})
df_merged = df_merged.sort_values(by=['patient', 'contact', 'amplitude'])
df_merged.to_csv('merged_massive.csv', index=False)
print(len(df_merged))
# Sort the DataFrame by 'mapping' in descending order, so that rows with 'mapping' = 1 appear first
df_merged = df_merged.sort_values('mapping', ascending=False)

# Drop duplicate rows based on 'massive_filename' and keep the first occurrence (which has 'mapping' = 1)
df_merged = df_merged.drop_duplicates(subset='massive_filename', keep='first')
df_merged = df_merged.sort_values(by=['patient', 'contact', 'amplitude'])
print(len(df_merged))
df_merged['part'] = np.nan
df_merged

15161
15161
14160


,center,cohort,lead_model,patient,lead,hemisphere,contact,vercise,directional,ring,segmented,amplitude,mapping_score,mapping,massive_filename,part
8173,Bern,B01,B.S. Vercise,1,1.0,right,0,0,False,R0,,1.0,NaN,0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c0_a1.nii,NaN
6215,Bern,B01,B.S. Vercise,1,1.0,right,0,0,False,R0,,1.5,NaN,0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c0_a1.5.nii,NaN
11407,Bern,B01,B.S. Vercise,1,1.0,right,0,0,False,R0,,2.0,NaN,0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c0_a2.nii,NaN
6404,Bern,B01,B.S. Vercise,1,1.0,right,0,0,False,R0,,2.5,NaN,0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c0_a2.5.nii,NaN
2219,Bern,B01,B.S. Vercise,1,1.0,right,0,0,False,R0,,3.0,NaN,0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c0_a3.nii,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3411,Bern,B01,B.S. Vercise,9,9.5,left,9,9,True,,R1_1,3.0,NaN,0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p009_c9_a3.nii,NaN
78,Bern,B01,B.S. Vercise,9,9.5,left,9,9,True,,R1_1,3.5,1.0,1,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p009_c9_a3.5.nii,NaN
14975,Bern,B01,B.S. Vercise,9,9.5,left,9,9,True,,R1_1,4.0,NaN,0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p009_c9_a4.nii,NaN
7960,Bern,B01,B.S. Vercise,9,9.5,left,9,9,True,,R1_1,4.5,NaN,0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p009_c9_a4.5.nii,NaN


In [69]:
df_merged['patient'].value_counts()

patient
27     300
149    300
18     300
23     300
26     300
      ... 
218     80
220     80
222     80
219     80
223     79
Name: count, Length: 78, dtype: int64

In [70]:

def calculate_part(group):
    #print(group)
    #print(f"{group['patientID'].iloc[0]}, {group['contactID'].iloc[0]}, {group['amplitude'].iloc[0]}")
    if group['mapping'].any() != 1:
        #print('empty group')
        return pd.DataFrame()  # Return an empty DataFrame if no 'mapping' value is found
    mapping_1_rows = group[group['mapping'] == 1]
    critical_amplitude = mapping_1_rows['amplitude'].max()
    partial = (group['amplitude'] <= critical_amplitude).astype(int)
    group['part'] = partial
    #print(critical_amplitude, partial)
    #print()
    return group

# Group by 'patientID', and 'contactID' and apply the calculate_part function
df_merged = df_merged.groupby(['patient', 'vercise']).apply(calculate_part).reset_index(drop=True)
print(len(df_merged))
df_merged['contact'] = df_merged['contact']
df_merged['mapping'] = df_merged['mapping']
df_merged['patient'] = df_merged['patient']

def filter_by_increment_of_point_five(row):
    fractional_part, _ = math.modf(float(row['amplitude']))
    #print(f"Fractional part: {fractional_part}")  # Debug line
    return math.isclose(fractional_part, 0.5, rel_tol=1e-9) or math.isclose(fractional_part, 0.0, rel_tol=1e-9)

df_merged = df_merged[df_merged.apply(filter_by_increment_of_point_five, axis=1)]
print(len(df_merged))
# patient, contact : (12, 12) = 4.25, (13, 10) = 2.75, (13, 17) = 4.25

8625
8622


In [71]:
df_merged['amplitude'].value_counts()

amplitude
1.0    780
1.5    780
2.0    780
2.5    780
3.0    780
3.5    780
4.0    780
4.5    780
5.0    780
5.5    247
6.0    247
6.5    247
7.0    247
7.5    247
8.0    247
0.5    120
Name: count, dtype: int64

In [72]:
def extended_interp(x, xp, fp):
    result = []
    for val in x:
        if val <= xp[-1]:
            result.append(np.interp(val, xp, fp))
        else:
            slope = (fp[-1] - fp[-2]) / (xp[-1] - xp[-2])
            result.append(fp[-1] + slope * (val - xp[-1]))
    return np.asarray(result)


def lin_interp_group(group):
    #print(group)
    # Find the indices where 'mapping' column is 1
    # Step 6: Sort the DataFrame by column E in descending order
    group = group.sort_values(by='mapping', ascending=False)
    # Step 7: Remove duplicate rows based on columns C, D, and E within each group
    group = group.drop_duplicates(subset=['patient', 'contact', 'amplitude'], keep='first')

    group = group.sort_values(by='amplitude', ascending=True)

    mask = group['mapping'] == 1
    indices = group.index[mask]

    if len(indices) == 0:
        # Return an empty DataFrame if no ground truth values found
        return pd.DataFrame()

    # Interpolate values based on ground truth points
    # Create an array of consecutive numbers as x-values
    x_values = group.loc[indices, 'amplitude'].astype(float).to_list()
    y_values = group.loc[indices, 'mapping_score'].astype(float).to_list()
    x_full = group.loc[:, 'amplitude'].astype(float).to_list()
    # Map the sorted indices to the corresponding x-values
    x_values.insert(0, 0)
    y_values.insert(0, 0)
    # Interpolate values based on ground truth points

    interpolated_values = extended_interp(x_full, x_values, y_values)

    # Replace interpolated values that exceed 1 with 1 and below 0 with 0
    interpolated_values = np.where(interpolated_values > 1, 1, interpolated_values)
    interpolated_values = np.where(interpolated_values < 0, 0, interpolated_values)

    # Assign interpolated values to the 'relativeImprovement' column
    group['lin_interp_score'] = interpolated_values

    return group

def step_interp_group(group):

    group = group.sort_values(by='mapping', ascending=False)
    group = group.drop_duplicates(subset=['patient', 'contact', 'amplitude'], keep='first')
    group = group.sort_values(by='amplitude', ascending=True)

    mask = group['mapping'] == 1
    indices = group.index[mask]

    if len(indices) == 0:
        # Return an empty DataFrame if no ground truth values found
        return pd.DataFrame()
    
    group['step_interp_score'] = group['mapping_score'].interpolate(method='pad')

    group['step_interp_score'] = group['step_interp_score'].fillna(0.0)

    return group


Missing : origin -> first GT point, last GT point -> same inter-slope

In [73]:

df_merged['lin_interp_score'] = np.nan
df_merged['step_interp_score'] = np.nan

df_merged = df_merged.groupby(['patient', 'contact']).apply(lin_interp_group).reset_index(drop=True)
df_merged = df_merged.groupby(['patient', 'contact']).apply(step_interp_group).reset_index(drop=True)
df_merged = df_merged.sort_values(by=['patient', 'contact', 'amplitude'])

df_merged.to_csv('/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/data/raw/tables/stn_space/merged/flipped/table.csv', index=False)
df_merged

,center,cohort,lead_model,patient,lead,hemisphere,contact,vercise,directional,ring,segmented,amplitude,mapping_score,mapping,massive_filename,part,lin_interp_score,step_interp_score
0,Bern,B01,B.S. Vercise,1,1.0,right,0,0,False,R0,,1.0,NaN,0.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c0_a1.nii,1.0,0.074074,0.0
1,Bern,B01,B.S. Vercise,1,1.0,right,0,0,False,R0,,1.5,NaN,0.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c0_a1.5.nii,1.0,0.111111,0.0
2,Bern,B01,B.S. Vercise,1,1.0,right,0,0,False,R0,,2.0,NaN,0.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c0_a2.nii,1.0,0.148148,0.0
3,Bern,B01,B.S. Vercise,1,1.0,right,0,0,False,R0,,2.5,NaN,0.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c0_a2.5.nii,1.0,0.185185,0.0
4,Bern,B01,B.S. Vercise,1,1.0,right,0,0,False,R0,,3.0,NaN,0.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p001_c0_a3.nii,1.0,0.222222,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8589,Bern,B01,B.S. Vercise,9,9.5,left,9,9,True,,R1_1,3.0,NaN,0.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p009_c9_a3.nii,1.0,0.857143,0.0
8590,Bern,B01,B.S. Vercise,9,9.5,left,9,9,True,,R1_1,3.5,1.0,1.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p009_c9_a3.5.nii,1.0,1.000000,1.0
8591,Bern,B01,B.S. Vercise,9,9.5,left,9,9,True,,R1_1,4.0,NaN,0.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p009_c9_a4.nii,0.0,1.000000,1.0
8592,Bern,B01,B.S. Vercise,9,9.5,left,9,9,True,,R1_1,4.5,NaN,0.0,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/04_MassiveMerged/p009_c9_a4.5.nii,0.0,1.000000,1.0


In [53]:
df_merged = pd.read_csv('/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/data/raw/tables/stn_space/merged/flipped/table.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/data/raw/tables/stn_space/merged/flipped/table.csv'

In [ ]:
df_merged.dtypes

In [ ]:
c1 = df_merged['patient'] == 12
c2 = df_merged['contact'] == 0
c3 = c1 & c2
df_merged[c3]

In [33]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Dropdown

# Grouping the dataframe
grouped_df = df_merged.groupby(['patient', 'contact'])

# Extracting the unique group indices
group_indices = grouped_df.groups.keys()
group_indices = sorted(group_indices)

@interact(group_index=IntSlider(min=0, max=len(group_indices)-1, step=1, value=0),
          score_type=Dropdown(options=['lin_interp_score', 'step_interp_score'], value='lin_interp_score'))
def plot_group(group_index, score_type):
    # Getting the corresponding group
    group = group_indices[group_index]
    data = grouped_df.get_group(group)

    # Extracting the selected score column
    score_column = data[score_type]

    # Setting colors for mapping_score and the selected score type
    mapping_color = 'red'
    score_color = 'blue' if score_type == 'lin_interp_score' else 'green'

    # Sorting the dataframe by amplitude
    data = data.sort_values(by='amplitude')

    # Plotting the data
    plt.figure(figsize=(10, 6))

    # Plotting all points with a semi-transparent line connecting them
    plt.plot(data['amplitude'].astype(float), score_column.astype(float), color='black', linestyle='-', alpha=0.3, zorder=1)

    # Plotting mapping_score points on top in red
    plt.scatter(data[data['mapping'] == 1]['amplitude'].astype(float), score_column[data['mapping'] == 1].astype(float),
                color=mapping_color, marker='o', label='Mapping', zorder=20)
    # Plotting selected score type points in blue or green
    plt.scatter(data[data['mapping'] != 1]['amplitude'].astype(float), score_column[data['mapping'] != 1].astype(float),
                color=score_color, marker='o', label=f'{score_type.capitalize()} Interpolation', zorder=2)

    # Plotting a line from the origin to the first point
    first_point_x = data['amplitude'].astype(float).iloc[0]
    first_point_y = score_column.astype(float).iloc[0]
    plt.plot([0, first_point_x], [0, first_point_y], color='black', linestyle='--', alpha=0.5, zorder=1)

    plt.xlabel('Amplitude')
    plt.ylabel(score_type.capitalize())
    plt.title(f'Amplitude vs {score_type.capitalize()} (Group: {group})')
    plt.ylim(-0.1, 1.1)  # Set the y-axis limits between -0.1 and 1.1

    # Adjusting the x-axis range
    x_min = max(0, data['amplitude'].astype(float).min() - 1)
    x_max = data['amplitude'].astype(float).max() + 1
    plt.xlim(0, 8.5)

    plt.grid(True, zorder=0)
    plt.legend()
    plt.show()


interactive(children=(IntSlider(value=0, description='group_index', max=776), Dropdown(description='score_type…

In [ ]:
df_merged.tail()

In [ ]:
# augmented step part
output_path = joinfile(output_folder, 'merged/flipped/table.csv')
print(output_path)
Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
df_merged.to_csv(output_path)

In [ ]:
import pandas as pd

table_path = '/media/brainstimmaps/DATA/2009_DeepMaps01/04_Source/01_Development/deepmaps/data/raw/tables/stn_space/merged/flipped/table.csv'
df = pd.read_csv(table_path)
df.index.to_numpy()

In [ ]:
df.columns